In [ ]:
import tensorflow as tf,pandas as pd, numpy as np
from tensorflow.keras import layers, models

In [ ]:
df = pd.read_csv("/content/data200.csv")

In [ ]:
df.head()

,a_avg,a_std,a_median,b_avg,b_std,b_median,label
0,0.000158,0.007114,0.0000,70.2150,0.035797,70.20,0
1,0.000180,0.003915,0.0000,69.2970,0.985366,69.60,0
2,0.000193,0.003369,0.0000,64.7645,2.513799,64.95,0
3,0.000206,0.006469,-0.0008,57.8365,2.564018,57.55,0
4,-0.000430,0.015124,-0.0008,53.1450,1.089362,52.80,0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channel dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class PrimaryCapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules):
        super(PrimaryCapsuleLayer, self).__init__()
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.dense_layers = [layers.Dense(units=dim_capsules, activation='relu') for _ in range(num_capsules)]

    def call(self, inputs):
        outputs = [dense_layer(inputs) for dense_layer in self.dense_layers]
        outputs = tf.stack(outputs, axis=1)
        return self.squash(outputs)

    def squash(self, s):
        s_norm = tf.norm(s, axis=-1, keepdims=True)
        scale = s_norm**2 / (1 + s_norm**2) / (s_norm + tf.keras.backend.epsilon())
        return scale * s

class DotProductLayer(layers.Layer):
    def __init__(self, axis=-1):
        super(DotProductLayer, self).__init__()
        self.axis = axis

    def call(self, inputs):
        return tf.reduce_sum(inputs[0] * inputs[1], axis=self.axis)

# Define input shape
input_shape = (6,)  # Each input sample is a vector of 6 features

# Create input layer
inputs = layers.Input(shape=input_shape)

# Instantiate Primary Capsule Layer
primary_caps = PrimaryCapsuleLayer(num_capsules=3,  # Number of capsules
                                   dim_capsules=6  # Dimensions of each capsule
                                  )(inputs)




In [ ]:
dense_layer = layers.Dense(units=6, activation='linear')(primary_caps)


In [ ]:
iterations = 3
for i in range(iterations):
  dot_product_layer = DotProductLayer(axis=-1)
  output = dot_product_layer([dense_layer, primary_caps])
  c_1_1 = tf.exp(output[0][0]) / (tf.exp(output[0][0]) + tf.exp(output[0][1]) + tf.exp(output[0][2]))
  c_2_1 = tf.exp(output[0][1]) / (tf.exp(output[0][0]) + tf.exp(output[0][1]) + tf.exp(output[0][2]))
  c_3_1 = tf.exp(output[0][2]) / (tf.exp(output[0][0]) + tf.exp(output[0][1]) + tf.exp(output[0][2]))


In [ ]:
dense_layer.shape,primary_caps.shape,output.shape

(TensorShape([None, 3, 6]), TensorShape([None, 3, 6]), TensorShape([None, 3]))

In [ ]:
# Build the model
model = models.Model(inputs=inputs, outputs=primary_caps)

# Print model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 6)]               0         
                                                                 
 primary_capsule_layer (Pri  (None, 3, 6)              126       
 maryCapsuleLayer)                                               
                                                                 
Total params: 126 (504.00 Byte)
Trainable params: 126 (504.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
